In [860]:
from collections import Counter, namedtuple
from gensim.models import KeyedVectors
from nltk.stem import WordNetLemmatizer
from pyemd import emd_with_flow
from sklearn.metrics import euclidean_distances

import time

In [861]:
from flow_wmd.modules import Document, DocPair

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [867]:
# Obama speaks to the media in Illinois 
sentence1_words = ['beautiful','obama', 'speaks', 'media', 'illinois', 'morning','Iraq']

# The President greets the press in Chicago. 
sentence2_words = ['beautiful','president', 'greets', 'press', 'chicago', 'beautiful','morning']

In [868]:
source = Document(sentence1_words)
source.instantiate(model)

sink = Document(sentence2_words)
sink.instantiate(model)

In [869]:
dp = DocPair(source, sink)
dp.getsignature()
dp.emd()
dp.getCost()

In [870]:
dp.source_cost

{'beautiful': 0.0,
 'obama': 0.19391,
 'speaks': 0.13973,
 'media': 0.12756,
 'illinois': 0.12296,
 'morning': 0.0,
 'Iraq': 0.18869}

In [871]:
dp.sink_cost

{'beautiful': 0.0,
 'president': 0.18869,
 'greets': 0.13973,
 'press': 0.12756,
 'chicago': 0.12296,
 'morning': 0.0}

In [874]:
w2v_emd, w2v_flow = dp.emd()

In [875]:
np.sum(np.array(w2v_flow)[np.ix_(dp.idx1,)], axis=1)

array([0.142857, 0.142857, 0.142857, 0.142857, 0.142857, 0.142857,
       0.142857])

In [876]:
dp.vocab

['beautiful',
 'obama',
 'speaks',
 'media',
 'illinois',
 'morning',
 'Iraq',
 'beautiful',
 'president',
 'greets',
 'press',
 'chicago',
 'beautiful',
 'morning']

In [878]:
import pandas as pd

flow_df = pd.DataFrame(w2v_flow, index=dp.vocab, columns=dp.vocab)
flow_df.iloc[dp.idx1,dp.idx2]

,beautiful,president,greets,press,chicago,beautiful,morning
beautiful,0.142857,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
obama,0.142857,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
speaks,0.000000,0.000000,0.142857,0.000000,0.000000,0.0,0.000000
media,0.000000,0.000000,0.000000,0.142857,0.000000,0.0,0.000000
illinois,0.000000,0.000000,0.000000,0.000000,0.142857,0.0,0.000000
morning,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.142857
Iraq,0.000000,0.142857,0.000000,0.000000,0.000000,0.0,0.000000


In [879]:
dist_df = pd.DataFrame(dp.w2v_distances, index=dp.vocab, columns=dp.vocab)
cost_df = (flow_df * dist_df)
cost_df.iloc[dp.idx1,dp.idx2].round(3)

,beautiful,president,greets,press,chicago,beautiful,morning
beautiful,0.000,0.000,0.00,0.000,0.000,0.0,0.0
obama,0.194,0.000,0.00,0.000,0.000,0.0,0.0
speaks,0.000,0.000,0.14,0.000,0.000,0.0,0.0
media,0.000,0.000,0.00,0.128,0.000,0.0,0.0
illinois,0.000,0.000,0.00,0.000,0.123,0.0,0.0
morning,0.000,0.000,0.00,0.000,0.000,0.0,0.0
Iraq,0.000,0.189,0.00,0.000,0.000,0.0,0.0


In [542]:
def read_1w_corpus(name, sep="\t"):
    for line in open(name):
        yield line.split(sep)

In [6]:
print("Loading GoogleNews Vectors")
%time model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
vocabulary = set(model.vocab)
relevant_words = [word for (word, count) in read_1w_corpus('count_1w.txt') if word in vocabulary]
model_reduced = model[[w for w in relevant_words]]

Loading GoogleNews Vectors


In [7]:
%time model.init_sims(replace=True)

CPU times: user 3.25 s, sys: 4.25 s, total: 7.5 s
Wall time: 8.49 s
